## Tek Terimden Çift Terime Geçiş 


In [1]:
import json
with open('trigger_request.json') as f:
    request_data = json.load(f)

### Step 1: Transition Process Control
- **State Number**: 2
- **State Code**: 10350002
- **State Name**: Transition Process from Single to Dual Term Initiated
- **Description**: This state controls the transition process. If all five checks are successful, it moves to the next state; otherwise, it goes to the termination state.

#### 1a. Çalışma Öncesi Script

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping
def state_handler(machine, state, request_data, context, state_response_of_machine):

    return {}



#### 1b. İstek Eşleştirme Scripti 

In [19]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping
def state_handler(machine, state, request_data, context, state_response_of_machine):
    ServicePointId = next((attr for attr in request_data['attributes'] if attr['attribute']['attributeCode'] == 'ServicePointId'), None)['value']
    return {"assetid":ServicePointId}

state_handler(None, None, request_data, None, None)

{'assetid': 21407}

#### 1c. State Context Değer Tutma Scripti

In [ ]:
# machine           : Currently executing machine instance
# state             : Currently executing state instance
# source            : Original trigger data
# context           : Machine stored context
# prevStateResponse : Previous executed state business logic response
# return
#     store dictionary  
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    ServicePointId = next((attr for attr in request_data['attributes'] if attr['attribute']['attributeCode'] == 'ServicePointId'), None)['value']

    store= {'ServicePointId':ServicePointId} 
    
    return store


{'ServicePointId': 21407}

#### 1d. Çalışma Sonrası Scripti

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping
def state_handler(machine, state, request_data, context, state_responses_of_machine):

    current_state_response = state_responses_of_machine['10350002']['businessLogicResponse']

    termType = next((attr for attr in request_data['attributes'] if attr['attribute']['attributeCode'] == 'TermType'), None)['values'][0]['value']
    cancelDate = next((attr for attr in request_data['attributes'] if attr['attribute']['attributeCode'] == 'ContractCancellationDate'), None)['value']

    
    if request_data['count'] > 3:
        return {"success": True, "error": {"code": "END", "detail": "Terim Sayısı"}}
    if cancelDate != 99991231000000:
        return {"success": True, "error": {"code": "END", "detail": "İptal Tarihi"}}
    if termType == 'ÇT':
        return {"success": True, "error": {"code": "END", "detail": "Terim Tipi"}}    
    if request_data['assetDefinitionCode'] not in ['ConsumerContract', 'LicensedProducerContract',
                                              'UnlicensedProducerContract']:
        return {"success": True, "error": {"code": "END", "detail": "Asset Definition"}}
    
    if request_data['assetDefinitionCode'] == 'ConsumerContract':
        tariff_code = next((attr['value'] for attr in current_state_response['attributes']
                            if attr['attribute']['attributeCode'] == 'VoltageType'), None)['values'][0]['value']

        # Eğer tariff_code None ise, values'dan al
        if tariff_code == 'AG':
                return {"success": True, "error": {"code": "END", "detail": "AG iseniz terim değişikliği yapılamaz"}}
        else : 
            return {"success": True, "goto": "10350003"}



state_handler(None, None, request_data, None, None)

## Step 2: Invoice Creation
- **State Number**: 3
- **State Code**: 10340003
- **State Name**: Invoice Creation
- **Description**: After the machine is created, it waits for the first invoice to be generated. At this stage, a specific event (from Kafka) is triggered.
- **Status**: Pending

#### 2a. Çalışma Öncesi Scripti

In [ ]:
#Yok

#### 2b. İstek Eşleştirme Scripti 

In [ ]:
#Yok

#### 2c. State Context Değer Tutma Scripti

In [ ]:
#Yok

#### 2d. Çalışma Sonrası Scripti

In [ ]:
#Yok

## Step 3: Contract Cancellation
- **State Number**: 4
- **State Code**: 10340004
- **State Name**: Contract Cancellation
- **Description**: This state signifies the Cancellation of the existing dual-term contract. Relevant date information is updated here, and then it transitions to the next state.

#### 3a. Çalışma Öncesi Scripti

In [ ]:
#Yok

#### 3b. İstek Eşleştirme Scripti 

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping
import json
from datetime import datetime

def state_handler(machine, state, request_data, context, state_responses_of_machine):
    
    current_date = datetime.now().strftime('%Y%m%d%H%M%S') 
    
    data = {
        "id": request_data["id"],
        "attributes": [
            {
                "attribute": {
                    "attributeCode": "ContractCancellationDate"
                },
                "value":current_date
            },
            {
                "attribute": {
                    "attributeCode": "ContractCancellationRequestDate"
                },
                "value": current_date
            },
            {
                "attribute": {
                    "attributeCode": "ContractCancellationReason"
                },
                "value": "SubscriberRequest"  
            },
            {
                "attribute": {
                    "attributeCode": "ContractNumber"
                },
                "value": next((attr['value'] for attr in request_data['attributes'] 
                               if attr['attribute']['attributeCode'] == "ContractNumber"), None)
            }
        ],
        "authorityGroup": request_data["authorityGroup"],
        "contractLockID": "string"  
    }
    
    return {"body": data}


{'id': 21408,
 'attributes': [{'attribute': {'attributeCode': 'ContractCancellationDate'},
   'value': '20241106000000'},
  {'attribute': {'attributeCode': 'ContractCancellationRequestDate'},
   'value': '20241106000000'},
  {'attribute': {'attributeCode': 'ContractCancellationReason'},
   'value': 'SubscriberRequest'},
  {'attribute': {'attributeCode': 'ContractNumber'}, 'value': 10000446}],
 'authorityGroup': 'string',
 'contractLockID': 'string'}

#### 3c. State Context Değer Tutma Scripti

In [ ]:
## Yok

#### 3d. Çalışma Sonrası Scripti

In [ ]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, state_response_of_machine):
    # do workflow here
    return {"success": True, "goto": "10350005"}



## Step 4: Closure Accrual Creation
- **State Number**: 5
- **State Code**: 10340005
- **State Name**: Closure Accrual Creation
- **Description**: A closure invoice is created for the terminated contract. Here, the invoice creation dates are processed.
- **Status**: Pending

#### 4a. Çalışma Öncesi Scripti

In [ ]:
# Yok

#### 4b. İstek Eşleştirme Scripti 

In [ ]:
#yok

#### 4c. State Context Değer Tutma Scripti

In [ ]:
#Yok

#### 4d. Çalışma Sonrası Scripti

In [ ]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, state_response_of_machine):
    # do workflow here
    return {"success": True, "goto": "10350006-1"}


#### 4e. İş Mantığı

In [ ]:
"""http://thor-stage.edas1.com/host/v1//accruals/calculate"""

'http://thor-stage.edas1.com/host/v1//accruals/calculate'

## Step 5: Data Preparation for New Account
- **State Number**: -
- **State Code**: -
- **State Name**: Data Preparation for New Account
- **Description**: This process involves preparing the necessary data for creating a new account. Relevant attributes are queried.
- **Status**: Pending

#### 5a. Çalışma Öncesi Scripti

In [ ]:
#Yok

#### 5b. İstek Eşleştirme Scripti 

In [ ]:
# machine                     : Currently executing machine instance
# state                       : Currently executing state instance
# request_data                : Original trigger data
# context                     : Machine stored context
# state_responses_of_machine  : State responses of Machine <StateCode, {prev, businessLogic, webhook}>
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, state_responses_of_machine):  
    contractAccountId = next((attr for attr in request_data['attributes'] if attr['attribute']['attributeCode']=='ContractAccountId'),None)['value']

    return {"params": {"assetId":contractAccountId}}

#### 5c. State Context Değer Tutma Scripti

In [ ]:
# machine           : Currently executing machine instance
# state             : Currently executing state instance
# source            : Original trigger data
# context           : Machine stored context
# prevStateResponse : Previous executed state business logic response
# return
#     store dictionary  
def state_handler(machine, state, request_data, context, state_response_of_machine):
    store = {'accountInfo' : state_response_of_machine['10350006-1']['businessLogicResponse']}
    return store

state_handler(None, None, request_data, None, None)

{'accountInfo': {'id': 21408,
  'assetDefinitionCode': 'LicensedProducerContract',
  'assetName': None,
  'authorityGroup': 'string',
  'assets': None,
  'count': 3,
  'attributes': [{'id': None,
    'attribute': {'alias': None, 'attributeCode': 'ReactiveExemption'},
    'value': None,
    'startDate': None,
    'endDate': None,
    'valueDesc': None,
    'values': [{'id': 13512,
      'attribute': None,
      'value': False,
      'startDate': 20241001000000,
      'endDate': 99991231235959,
      'valueDesc': None,
      'values': None}]},
   {'id': 16057,
    'attribute': {'alias': None, 'attributeCode': 'ContractNumber'},
    'value': 10000446,
    'startDate': None,
    'endDate': None,
    'valueDesc': None,
    'values': None},
   {'id': 13071,
    'attribute': {'alias': None,
     'attributeCode': 'ContractCancellationRequestDate'},
    'value': 99991231235959,
    'startDate': None,
    'endDate': None,
    'valueDesc': None,
    'values': None},
   {'id': 13070,
    'attribut

#### 5d. Çalışma Sonrası Scripti

In [ ]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, prev_state_response, current_state_response):
    # do workflow here
    return {"success": True, "goto": "10350006"}


#### 5e. İş Mantığı

In [ ]:
"""http://thor-stage.edas1.com/host/v1//md/search/{assetId}"""##URL is  not defined

'http://thor-stage.edas1.com/host/v1//md/search/{assetId}'

## Step 6: Create New Account
- **State Number**: 6
- **State Code**: 10340006
- **State Name**: Create New Account
- **Description**: An account is created to link with the new contract. New account definitions are made using existing attributes.
- **Status** : Pending

#### 6a. Çalışma Öncesi Scripti

In [ ]:
#Yok

#### 6b. İstek Eşleştirme Scripti 

In [ ]:
from datetime import datetime
import json
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    # Not recursive

    deletelist = []

    current_date = datetime.now().strftime('%Y%m%d%H%M%S') 
 
    copied_json = {}
    stack = [(copied_json, state_responses_of_machine['10340006-1']['businessLogicResponse'])]
 
    # Copy context to copied_json without id
    while stack:
        current_copied, current_original = stack.pop()
 
        for key, value in current_original.items():
            if key != 'id':
                if isinstance(value, list):
                    new_list = []
                    for item in value:
                        if isinstance(item, dict):
                            new_item = {}
                            new_list.append(new_item)
                            stack.append((new_item, item))
                        else:
                            new_list.append(item)
                    current_copied[key] = new_list
                elif isinstance(value, dict):
                    new_dict = {}
                    current_copied[key] = new_dict
                    stack.append((new_dict, value))
                else:
                    current_copied[key] = value
 
    # Set dates for all values in copied_json
    for key, value in copied_json.items():
        if isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    if item.get('value') is None and 'values' in item and item['values']:
                        item['values'][0]['startDate'] = current_date
                        item['values'][0]['endDate'] = 99991231235959
                    elif item.get('value') is not None:
                        item['startDate'] = current_date
                        item['endDate'] = 99991231235959
        
        elif isinstance(value, dict):
            if value.get('value') is None and 'values' in value and value['values']:
                value['values'][0]['startDate'] = current_date
                value['values'][0]['endDate'] = 99991231235959
            elif value.get('value') is not None:
                value['startDate'] = current_date
                value['endDate'] = 99991231235959
 
   
    for attr in copied_json['attributes']:
        if attr['attribute']['attributeCode']=='ExpiryDate':
            attr['value'] = current_date

    copied_json['attributes'] = [attr for attr in copied_json['attributes'] if attr['attribute']['attributeCode'] not in deletelist]

   
                   
 
    return {"body":copied_json}  
 


    
    


{'assetDefinitionCode': 'LicensedProducerContract',
 'assetName': None,
 'authorityGroup': 'string',
 'assets': None,
 'count': 3,
 'attributes': [{'attribute': {'alias': None,
    'attributeCode': 'ReactiveExemption'},
   'value': None,
   'startDate': None,
   'endDate': None,
   'valueDesc': None,
   'values': [{'attribute': None,
     'value': False,
     'startDate': 20241001000000,
     'endDate': 99991231235959,
     'valueDesc': None,
     'values': None}]},
  {'attribute': {'alias': None, 'attributeCode': 'ContractNumber'},
   'value': 10000446,
   'startDate': None,
   'endDate': None,
   'valueDesc': None,
   'values': None},
  {'attribute': {'alias': None,
    'attributeCode': 'ContractCancellationRequestDate'},
   'value': 99991231235959,
   'startDate': None,
   'endDate': None,
   'valueDesc': None,
   'values': None},
  {'attribute': {'alias': None, 'attributeCode': 'OldSystemContractBeginDate'},
   'value': None,
   'startDate': None,
   'endDate': None,
   'valueDesc'

#### 6c. State Context Değer Tutma Scripti

In [ ]:
# machine           : Currently executing machine instance
# state             : Currently executing state instance
# source            : Original trigger data
# context           : Machine stored context
# prevStateResponse : Previous executed state business logic response
# return
#     store dictionary  
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    store = {}
    store ['NewAccountId'] = state_responses_of_machine['10340006']['businessLogicResponse']['id'] 
    return store


#### 6d. Çalışma Sonrası Scripti

In [ ]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    # do workflow here
    return {"success": True, "goto": "10340007"}


#### 6e. İş Mantığı

In [ ]:
"""http://thor-stage.edas1.com/host/v1//account/create"""


'http://thor-stage.edas1.com/host/v1//account/create'

## Step 7: Creation of New Contract
- **State Number**: 7
- **State Code**: 10340007
- **State Name**: Creation of New Contract
- **Description**: A new contract is created with the new term information. The same attributes as the existing contract are used to create a single-term contract.

#### 7a. Çalışma Öncesi Scripti

In [ ]:
#Yok

##### 7b. İstek Eşleştirme Scripti- Olması gereken(!1)

In [ ]:
from datetime import datetime
import json
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    # Not recursive 
    current_date = datetime.now().strftime('%Y%m%d%H%M%S') 
    
    
    changes_dict = {'TermType': 'CT',
                    'ContractAccountId': context['accountInfo']['id']
                    }
    
    copied_json = {}
    stack = [(copied_json, request_data)]

    # Copy context to copied_json without id
    while stack:
        current_copied, current_original = stack.pop()

        for key, value in current_original.items():
            if key != 'id':
                if isinstance(value, list):
                    new_list = []
                    for item in value:
                        if isinstance(item, dict):
                            new_item = {}
                            new_list.append(new_item)
                            stack.append((new_item, item))
                        else:
                            new_list.append(item)
                    current_copied[key] = new_list
                elif isinstance(value, dict):
                    new_dict = {}
                    current_copied[key] = new_dict
                    stack.append((new_dict, value))
                else:
                    current_copied[key] = value

    # Set dates for all values in copied_json
    for key, value in copied_json.items():
        if isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    if item.get('value') is None and 'values' in item and item['values']:
                        item['values'][0]['startDate'] = current_date
                        item['values'][0]['endDate'] = 99991231235959
                    elif item.get('value') is not None:
                        item['startDate'] = current_date
                        item['endDate'] = 99991231235959
        elif isinstance(value, dict):
            if value.get('value') is None and 'values' in value and value['values']:
                value['values'][0]['startDate'] = current_date
                value['values'][0]['endDate'] = 99991231235959
            elif value.get('value') is not None:
                value['startDate'] = current_date
                value['endDate'] = 99991231235959

    # Update attribute values in context according to the dict above
    for code, new_value in changes_dict.items():
        for attr in copied_json.get('attributes', []):
            if attr['attribute']['attributeCode'] == code:
                if attr['value'] is None:
                    attr['values'][0]['value'] = new_value
                    attr['values'][0]['valueDesc'] = None
                else:
                    attr['valueDesc'] = None
                    attr['value'] = new_value

    return {"body":copied_json}  

{'assetDefinitionCode': 'LicensedProducerContract',
 'assetName': None,
 'authorityGroup': 'string',
 'assets': None,
 'count': 3,
 'attributes': [{'attribute': {'alias': None,
    'attributeCode': 'ReactiveExemption'},
   'value': None,
   'startDate': None,
   'endDate': None,
   'valueDesc': None,
   'values': [{'attribute': None,
     'value': False,
     'startDate': '20241106000000',
     'endDate': 99991231235959,
     'valueDesc': None,
     'values': None}]},
  {'attribute': {'alias': None, 'attributeCode': 'ContractNumber'},
   'value': 10000446,
   'startDate': '20241106000000',
   'endDate': 99991231235959,
   'valueDesc': None,
   'values': None},
  {'attribute': {'alias': None,
    'attributeCode': 'ContractCancellationRequestDate'},
   'value': 99991231235959,
   'startDate': '20241106000000',
   'endDate': 99991231235959,
   'valueDesc': None,
   'values': None},
  {'attribute': {'alias': None, 'attributeCode': 'OldSystemContractBeginDate'},
   'value': None,
   'startD

#### 7c. State Context Değer Tutma Scripti

In [ ]:
# machine           : Currently executing machine instance
# state             : Currently executing state instance
# source            : Original trigger data
# context           : Machine stored context
# prevStateResponse : Previous executed state business logic response
# return
#     store dictionary  
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    store = {}
    store ['NewContractId'] = state_responses_of_machine['1035007']['businessLogicResponse']['id'] 
    return store


##### 7d. Çalışma Sonrası Scripti

In [ ]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, state_response_of_machine):
    # do workflow here
    return {"success": True, "goto": "END"}


#### 7e. İş Mantığı

In [ ]:
"""http://thor-stage.edas1.com/host/v1//contract/create"""

'http://thor-stage.edas1.com/host/v1//contract/create'